#### 그레디언트 부스팅 계열의 앙상블 모델
- 대표 모델 : XGBoost, lightGBM
- 추가 설치 및 사용법 숙지 필요

- 설치 여부 확인

In [46]:
import xgboost, lightgbm

print(f'xgboost : {xgboost.__version__}')
print(f'lightgbm : {lightgbm.__version__}')

xgboost : 1.7.3
lightgbm : 4.1.0


[1] 모듈 로딩 및 데이터 준비 <hr>

In [47]:
## 데이터 파일 변수 선언
feature_name_file = '../data/human/features.txt'
label_file = '../data/human/activity_labels.txt'

XtrnFile = '../data/human/train/X_train.txt'
ytrnFile = '../data/human/train/y_train.txt'
XtstFile = '../data/human/test/X_test.txt'
ytstFile = '../data/human/test/y_test.txt'

In [48]:
#데이터 로딩
import pandas as pd
import numpy as np

featureName = pd.read_csv(feature_name_file, sep='\s+', header = None, names =['col_id', 'col_name']) #'\s+' 뭔진 모르겠지만 공백으로 나누는 건듯..?

In [49]:
# 데이터 확인
featureName.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 561 entries, 0 to 560
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   col_id    561 non-null    int64 
 1   col_name  561 non-null    object
dtypes: int64(1), object(1)
memory usage: 8.9+ KB


In [50]:
# 중복값 확인
featureName['col_name'].duplicated().sum()
# => 중복값 84개... ㄷ ㄷ 엄청 많다.

84

In [51]:
# 중복값만 필터링 --> 인덱스 추출 -> 리스트에 담기
dropIdx = featureName[featureName['col_name'].duplicated(keep='first')].index

In [56]:
featureName = pd.read_csv(feature_name_file, sep='\s+', header = None)

# 학습 데이터 로딩.
XtrnDF = pd.read_csv(XtrnFile, sep='\s+', header = None)
XtstDF = pd.read_csv(XtstFile, sep='\s+', header = None)
XtrnDF.info()

# df = pd.DataFrame({'label' : label_file,
#                     'featureName' : feature_name_file,
#                     'Xtrain' : XtrnFile
#                    }, index=range(0, len(label_file)))


#df

#for idx in idxlist :
    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 561 entries, 0 to 560
dtypes: float64(561)
memory usage: 31.5 MB


In [57]:
XtrnDF = XtrnDF.drop(dropIdx, axis='columns')
XtrnDF= XtrnDF.T.reset_index(drop=True).T # 컬럼을 지우고 새로 인덱스를 부여하기 위해서 한번 돌린 후에 인덱스 새로 부여 ▶ 다시 원상복귀
print(XtrnDF.info())
print()
XtstDF = XtstDF.drop(dropIdx, axis='columns')
XtstDF= XtstDF.T.reset_index(drop=True).T
print(XtstDF.info())
print()
featureName = featureName.drop(dropIdx)
featureName= featureName.reset_index(drop=True)
print(featureName.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7352 entries, 0 to 7351
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 26.8 MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2947 entries, 0 to 2946
Columns: 477 entries, 0 to 476
dtypes: float64(477)
memory usage: 10.7 MB
None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 477 entries, 0 to 476
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       477 non-null    int64 
 1   1       477 non-null    object
dtypes: int64(1), object(1)
memory usage: 7.6+ KB
None


In [54]:
featureName[0].duplicated().sum()

0

xgboost => 데이터형을 dmatrix로 바꿔야 함.